In [24]:
import json
import gpflow
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline
import seaborn as sns
sns.set_style("white")
sns.set_context("paper")
import sys
sys.path.append('../')
from Misc import syn_data_gen, save_load_files

# Load Actions Data of specific occupants

In [25]:
num_data = 20

In [26]:
# Specific occupants 6, 7, 8 actions
specific_occ_actions_data_list = []
for i in xrange(5,8):
    occ_num = i + 1
    print "Occupant number:"
    print occ_num
    config_file = '../data/config_files/syn/diff_utilities_config/syn_occupants.json'
    with open(config_file, 'r') as fd:
        config = json.loads(fd.read())

    a = config['O' + str(occ_num)]['a']
    b = config['O' + str(occ_num)]['b']
    
    # Load actions data
    actions_data = np.loadtxt('../data/syn_data/syn_occ/actions_data/train/' +
                         'O_a' + str(a) + '_b' + str(b) + '_' + str(num_data) + '_.csv', delimiter = ',')
    
    specific_occ_actions_data_list.append(actions_data)

Occupant number:
6
Occupant number:
7
Occupant number:
8


In [27]:
all_x_grid = np.linspace(0,1,50)
all_x_grid_norm = (all_x_grid - 0.5)/0.5

In [28]:
def gen_req_action_data(action_data_matrix, X_grid):
    """
    Inputs:
    actions_data_matrix : action data matrix (X_prev, X_current)
    Outputs:
    Concatenation of [X_cur, X_prev]
    Index of action data in X_grid
    """
    prev = action_data_matrix[:,0][:,None]
    cur = action_data_matrix[:,1][:,None]
    concat = np.vstack([cur, prev])
    
    grid_sorted = np.argsort(X_grid)
    ypos = np.searchsorted(X_grid[grid_sorted], concat)
    indices = grid_sorted[ypos]
    return concat, indices

In [29]:
concat_actions, indices = gen_req_action_data(specific_occ_actions_data_list[0], all_x_grid)

In [11]:
def gen_rel_list_action_data(all_occ_action_data_list, X_grid):
    """
    Generate relevant data from all occupant action data list
    """
    
    concat_list = []
    indices_list = []
    for action_data in all_occ_action_data_list:
        concat_actions, indices = gen_req_action_data(action_data, X_grid)
        concat_list.append(concat_actions)
        indices_list.append(indices)
        
    return concat_list, indices_list

In [18]:
def gen_rel_matrix_action_data(all_occ_action_data_list, X_grid):
    """
    Generate relevant data from all occupant action data list
    """
    
    same_const_shape = 2*all_occ_action_data_list[0].shape[0]
    concat_mat = np.zeros(shape = [len(all_occ_action_data_list), same_const_shape])
    indices_mat = np.zeros(shape = [len(all_occ_action_data_list), same_const_shape])
    for i, action_data in enumerate(all_occ_action_data_list):
        concat_actions, indices = gen_req_action_data(action_data, X_grid)
        concat_mat[i,:] = concat_actions[:,0]
        indices_mat[i,:] = indices[:,0]
        
    return concat_mat, indices_mat

In [19]:
concat_list, indices_list = gen_rel_list_action_data(specific_occ_actions_data_list, all_x_grid)

In [20]:
concat_mat, indices_mat = gen_rel_matrix_action_data(specific_occ_actions_data_list, all_x_grid)

In [22]:
concat_mat.shape

(3, 40)

In [23]:
indices_mat.shape

(3, 40)